In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
select_neurons = ['Neuron_1', 'Neuron_2', 
                  'Neuron_5', 'Neuron_6', 'Neuron_7']

spike_inf = pd.read_csv("/media/ubuntu/sda/data/mouse5/output/analysis_natural_image/02_merge_new/spike_inf.tsv", sep = '\t')
cluster_inf = pd.read_csv("/media/ubuntu/sda/data/mouse5/output/analysis_natural_image/02_merge_new/cluster_inf.tsv", sep = '\t')

In [7]:
trigger_time = pd.read_csv("/media/ubuntu/sda/data/mouse5/output/analysis_natural_image/01_get_trigger/trigger_time.tsv", sep='\t')

import pickle
with open('/media/ubuntu/sda/data/mouse5/output/analysis_natural_image/02_merge_new/outlier.pkl', 'rb') as f:
    outlier = pickle.load(f)


In [8]:
for i in outlier:
    date = i.split("_")[0]
    image = i.split("_")[1] + "_" + i.split("_")[2]
    trigger_time = trigger_time[~((trigger_time['date'] == int(date)) & (trigger_time['order'] == image))]

In [11]:
# expand time to -0.5s - 1.5s
trigger_time['expand_start'] = trigger_time['start'] - 5000
trigger_time['expand_end'] = trigger_time['end'] + 10000


In [12]:
date_order = ['030222', '042422', '052322', '062322', '082422', '092222','102522', '112822', '122322', '012123', '022423', '032323', '042323',
              '052423', '062323', '072123']

In [13]:
with PdfPages("raster_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        fig, ax = plt.subplots(figsize = (12, 7.5))

        for index, row in trigger_time.iterrows():
            date = row['date']
            image_order = row['order']
            start = row['expand_start'] + 2500
            end = row['end'] -2500
            
            filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                        (neuron_df['time'] >= start) & 
                                        (neuron_df['time'] <= end)]
            
            if not filtered_spikes.empty:
                ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), marker='|', mew=1, markersize=3, ls='', color='k')

        ax.set_yticks([])
        #ax.axvspan(1 + (5000 - 1) * 0.5, 1 + (5000 - 1), color='gray', alpha=0.3)

        ax.set_xlabel('time (s)')
        ax.set_ylabel("")
        ax.set_title(f'Raster Plot for {neuron}')

        pdf.savefig(fig)
        plt.close()

In [14]:
with PdfPages("peth_lineplot_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        fig, ax = plt.subplots(figsize=(10, 6))

        dates = trigger_time['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates = []
        std_rates = []

        for date in dates:
            trigger_time_temp = trigger_time[trigger_time['date'] == date]
            all_spikes = []

            for index, row in trigger_time_temp.iterrows():
                start = row['expand_start'] + 2500
                end = row['end'] -2500

                filtered_spikes = spike_inf[(spike_inf['Neuron'] == neuron) &
                                            (spike_inf['date'] == date) &
                                            (spike_inf['time'] >= start) &
                                            (spike_inf['time'] <= end)]

                if not filtered_spikes.empty:
                    relative_spikes = filtered_spikes['time'] - start
                    all_spikes.extend(relative_spikes)

            if all_spikes:
                time_bins = np.linspace(0, max(all_spikes), 100)
                spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                spike_rates.append(spike_rate)

        if spike_rates:
            mean_spike_rate = np.mean(spike_rates, axis=0)
            std_spike_rate = np.std(spike_rates, axis=0)

            ax.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth=2, label='Mean Spike Rate')
            ax.fill_between(time_bins[:-1], mean_spike_rate - std_spike_rate, mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.5, label='Std Deviation')

            ax.set_title(f"Neuron {neuron} - Overall Mean PSTH", fontsize=14)
            ax.set_xlabel('Time (s)', fontsize=12)
            ax.set_ylabel('Spike Rate (Hz)', fontsize=12)
            ax.legend()

        pdf.savefig(fig)
        plt.close()

In [15]:
with PdfPages("raster_split_image.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]


        fig, axs = plt.subplots(9, 13, figsize=(25, 15), sharex=True, sharey=True)
        fig.subplots_adjust(hspace=0.5, wspace=0.5)

        for image in range(1, 118):  
            ax = axs[(image - 1) // 13, (image - 1) % 13]  
            trigger_time_temp = trigger_time[trigger_time['image'] == image]

            for index, row in trigger_time_temp.iterrows():
                date = row['date']
                image_order = row['order']
                start = row['expand_start'] + 2500
                end = row['end'] - 2500

                filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                            (neuron_df['time'] >= start) & 
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), 
                            marker='|', mew=1, markersize=3, ls='', color='k')


            ax.set_xticks([])
            ax.set_yticks([])

        pdf.savefig(fig)
        plt.close()

In [16]:
mean_spike_rate_dict = {}

with PdfPages("peth_lineplot.pdf") as pdf:
    for image in range(1, 118):
        fig, axes = plt.subplots(4, 5, figsize=(15, 9))
        axes = axes.flatten()
        num = 0
        image_mean_spike_rate_data = []
        for neuron in spike_inf['Neuron'].unique():
            ax = axes[num]
            neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

            dates = trigger_time[trigger_time['image'] == image]['date'].unique()
            date_indices = np.arange(len(dates))  

            spike_rates = []
            std_rates = []

            for date in dates:
                trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
                all_spikes = []

                for index, row in trigger_time_temp.iterrows():
                    start = row['expand_start']
                    end = row['expand_end']

                    filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                                (neuron_df['time'] >= start) &
                                                (neuron_df['time'] <= end)]

                    if not filtered_spikes.empty:
                        relative_spikes = filtered_spikes['time'] - start
                        all_spikes.extend(relative_spikes)

                if all_spikes:
                    time_bins = np.linspace(0, max(all_spikes), 100)
                    spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                    spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                    spike_rates.append(spike_rate)

            mean_spike_rate = np.mean(spike_rates, axis=0)
            std_spike_rate = np.std(spike_rates, axis=0)

            image_mean_spike_rate_data.append({
                'Neuron': neuron,
                'Mean_Spike_Rate': mean_spike_rate
            })

            ax.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth = 2)
            ax.plot(time_bins[:-1], mean_spike_rate - std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
            ax.plot(time_bins[:-1], mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
            #ax.set_ylim(- np.mean(mean_spike_rate) , 2 *np.mean(mean_spike_rate))
            #ax.fill_between(time_bins[:-1], mean_spike_rate - std_spike_rate, mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.5, label='Std Deviation')
            quarter = (time_bins[-1] - time_bins[0]) * 0.25 + time_bins[0]
            three_quarter = (time_bins[-1] - time_bins[0]) * 0.75 + time_bins[0]
            ax.annotate("", xy=(quarter, np.max(mean_spike_rate + std_spike_rate) * 0.9), xytext=(quarter, np.max(mean_spike_rate + std_spike_rate)),
                        arrowprops=dict(arrowstyle="-|>", color="black"))
            ax.annotate("", xy=(three_quarter, np.max(mean_spike_rate + std_spike_rate) * 0.9), xytext=(three_quarter, np.max(mean_spike_rate + std_spike_rate)),
                        arrowprops=dict(arrowstyle="-|>", color="black"))
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.set_title(f'{neuron}', fontsize=10)
            ax.legend()

            ax.set_xticks([])
            ax.set_yticks([])
            ax.grid(False)
            ax.legend().set_visible(False)
            num += 1
        axes[19].axis('off')
        axes[18].axis('off')
        fig.suptitle(f"Image_{image}", fontsize=20)
        fig.text(0.5, 0.04, 'Time', ha='center', fontsize=14)
        fig.text(0.06, 0.5, 'Spike Rate (Hz)', va='center', rotation='vertical', fontsize=14)
        pdf.savefig(fig)
        plt.close()

        mean_spike_rate_dict[image] = pd.DataFrame(image_mean_spike_rate_data)

In [17]:
trigger_time.to_csv("trigger_time_refine.csv")